# Atención primaria España

In [1]:
import pandas as pd
import os

datos [porta estadístico](https://pestadistico.inteligenciadegestion.mscbs.es/publicoSNS/S/base-de-datos-de-clinicos-de-atencion-primaria-bdcap)

In [2]:
#consulta de datasets

In [3]:
# os.listdir('../data/at_primaria')
#'at_primaria.csv' = at_prim.xlsx

In [4]:
psico_prim = pd.read_excel('../data/at_primaria/detalles_psico.xlsx')

In [5]:

psico_prim.columns = psico_prim.columns.str.replace(" ",'_')

In [6]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Grandes_Grupos,Problema_de_Salud,Casos
95432,2015,15-Navarra,Mujer,Sin Clasificar,00-14 años,P85 - Retraso mental,NaN
25904,2013,05-Canarias,Mujer,?100.000 ? /año,00-14 años,P76 - Depresión/trastornos depresivos,NaN
196399,2019,07-Castilla y León,Mujer,Muy baja,35-64 años,P71 - Otras psicosis orgánicas,NaN
170506,2018,09-Cataluña,Mujer,Muy baja,00-14 años,P07 - Disminución del deseo sexual,NaN
139320,2017,08-Castilla la Mancha,Mujer,18.000-99.999 ? /año,15-34 años,P17 - Abuso del tabaco,NaN


In [7]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Grandes_Grupos            0
Problema_de_Salud         0
Casos                182993
dtype: int64

In [8]:
psico_prim.shape

(213840, 7)

In [9]:
psico_prim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213840 entries, 0 to 213839
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Año                213840 non-null  int64  
 1   CCAA               213840 non-null  object 
 2   Sexo               213840 non-null  object 
 3   Nivel_Renta        213840 non-null  object 
 4   Grandes_Grupos     213840 non-null  object 
 5   Problema_de_Salud  213840 non-null  object 
 6   Casos              30847 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 11.4+ MB


In [10]:
list(psico_prim.CCAA.unique())

['02-Aragón',
 '04-Baleares',
 '05-Canarias',
 '07-Castilla y León',
 '09-Cataluña',
 '12-Galicia',
 '16-País Vasco',
 '08-Castilla la Mancha',
 '10-Comunidad Valenciana',
 '15-Navarra',
 '03-Asturias',
 '06-Cantabria',
 '11-Extremadura',
 '13-Madrid',
 '14-Murcia',
 '01-Andalucía',
 '17-La Rioja']

In [11]:
#psico_prim.CCAA=psico_prim.CCAA.str.extract(r'([A-Z][a-z].*)')

In [12]:
psico_prim[["CodCA","CCAA"]] = psico_prim.CCAA.str.split("-",expand=True)

In [13]:
list(psico_prim.Nivel_Renta.unique())

['Sin Clasificar',
 '?100.000 ? /año',
 '18.000-99.999 ? /año',
 '<18.000 ? /año',
 'Muy baja']

In [14]:
renta ={'Sin Clasificar':'desconocida',
 '?100.000 ? /año': '+ 100000',
 '18.000-99.999 ? /año': '18000-99999',
 '<18.000 ? /año':'- 18000',
 'Muy baja':'muy_baja'}

In [15]:
psico_prim.Nivel_Renta = psico_prim.Nivel_Renta.replace(renta)

In [16]:
psico_prim.Grandes_Grupos = psico_prim.Grandes_Grupos.str.extract(r'(\d+-*\d+)')
psico_prim.rename({"Grandes_Grupos":"Edad"},axis = 1,inplace = True)
psico_prim.Edad.replace('65','65 +', inplace = True)

In [17]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Problema_de_Salud,Casos,CodCA
107450,2016,Castilla y León,Hombre,+ 100000,00-14,P03 - Sensación/sentimientos depresivos,NaN,07
73574,2015,Aragón,Mujer,- 18000,00-14,P07 - Disminución del deseo sexual,NaN,02
61195,2014,Extremadura,Mujer,18000-99999,00-14,P79 - Fobia/trastorno compulsivo,NaN,11
26781,2013,Cantabria,Hombre,+ 100000,00-14,P73 - Psicosis afectivas,NaN,06
144889,2017,Extremadura,Mujer,desconocida,35-64,P86 - Anorexia nerviosa/bulimia,NaN,11


In [18]:
psico_prim.Casos.unique()

array([ 284.,   nan, 1305., ...,  159.,   91., 4646.])

In [19]:
psico_prim.Casos.fillna(0.,inplace = True)

In [20]:
psico_prim[["cod","diag"]]= psico_prim.Problema_de_Salud.str.split(" - ",expand=True)

In [21]:
psico_prim.groupby(["Año","CCAA","Problema_de_Salud"]).agg({"Casos":"sum"}).reset_index()

,Año,CCAA,Problema_de_Salud,Casos
0,2011,Aragón,P01 - Sensación de ansiedad/tensión/nerviosismo,122032.0
1,2011,Aragón,P02 - Estrés agudo,2976.0
2,2011,Aragón,P03 - Sensación/sentimientos depresivos,6784.0
3,2011,Aragón,P04 - Sensación/sentimientos de irritabilidad/...,2207.0
4,2011,Aragón,P05 - Sentimientos/conducta senil,0.0
...,...,...,...,...
5759,2019,País Vasco,P82/02 - Reacción de adaptación no incluida en...,138393.0
5760,2019,País Vasco,P85 - Retraso mental,1609.0
5761,2019,País Vasco,P86 - Anorexia nerviosa/bulimia,847.0
5762,2019,País Vasco,P98 - Otras psicosis no especificadas,3939.0


In [22]:
psico_prim.Casos.max()

307385.0

In [23]:
capV = {'F00-F09': 'Trastornos mentales orgánicos, incluidos los trastornos sintomáticos',
 'F10-F19': 'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos',
 'F20-29': 'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes',
 'F30-39': 'Trastornos del estado de animo, trastornos afectivos',
 'F40-49': 'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos',
 'F50-59': 'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos',
 'F60-69': 'Trastornos de la personalidad y del comportamiento en adultos',
 'F70-79': 'Retraso mental',
 'F80-89': 'Trastornos del desarrollo psicológico',
 'F90-F98': 'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia',
 'F99': 'Trastornos mentales sin especificar'}

In [24]:
trans_cod = {'P01': 'F40-49','P02': 'F40-49','P03': 'F40-49','P04': 'F40-49','P05':'F00-F09','P06': 'F30-39','P07': 'F30-39',
'P08': 'F30-39','P09': 'F30-39','P09-07': 'F30-39','P10': 'F90-F98','P11': 'F90-F98','P12': 'F90-F98','P13': 'F90-F98',
'P15': 'F10-F19','P16': 'F10-F19','P17': 'F10-F19','P18': 'F10-F19','P19': 'F10-F19','P20':'F00-F09','P22+23': 'F90-F98',
'P24': 'F80-89','P25': 'F60-69','P27': 'F99','P28': 'F70-79','P29': 'F99','P70':'F00-F09','P71': 'F20-29','P72': 'F20-29',
'P73': 'F20-29','P74': 'F40-49','P75': 'F40-49','P76': 'F30-39','P77': 'F30-39','P78': 'F40-49','P79': 'F40-49','P80': 'F60-69',
'P81': 'F90-F98','P82': 'F40-49','P82/02': 'F40-49','P85': 'F70-79','P86': 'F50-59','P98': 'F20-29','P99': 'F99',
}

In [25]:
psico_prim['cei10'] = psico_prim.cod.replace(trans_cod)

In [26]:
psico_prim["cei10diag"] = psico_prim.cei10.replace(capV)

In [27]:
psico_prim.drop('Problema_de_Salud',axis=1,inplace=True)

In [28]:
psico_prim.Casos =psico_prim.Casos.astype('int64')

In [29]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod,diag,cei10,cei10diag
26847,2013,Cantabria,Hombre,+ 100000,35-64,0,06,P08,Ausencia/pérdida de la satisfación sexual,F30-39,"Trastornos del estado de animo, trastornos afe..."
141745,2017,Comunidad Valenciana,Hombre,+ 100000,35-64,0,10,P24,Problemas específicos del aprendizaje,F80-89,Trastornos del desarrollo psicológico
206270,2019,Madrid,Mujer,18000-99999,00-14,0,13,P98,Otras psicosis no especificadas,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra..."
201977,2019,Extremadura,Hombre,18000-99999,15-34,0,11,P18,Abuso de fármacos,F10-F19,Trastornos mentales y de comportamiento debido...
84256,2015,Cataluña,Hombre,- 18000,15-34,681,09,P85,Retraso mental,F70-79,Retraso mental


In [30]:
psico_prim= psico_prim.iloc[:, [0,1,6,2,3,4,7,8,9,10,5]]  

In [31]:
psico_prim.to_csv('../data/psico_atprimaria.csv',index=False)